In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

from catboost import CatBoostClassifier, cv, Pool
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, auc, roc_auc_score
from dmba import classificationSummary

from imblearn.over_sampling import SMOTE

In [2]:
train_df = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Train.csv')
test_df = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Test.csv')

train_df['ownerChange'] = round(train_df['ownerChange'])
train_df['bedCount_class'] = round(train_df['bedCount_class'])

test_df['ownerChange'] = round(train_df['ownerChange'])
test_df['bedCount_class'] = round(train_df['bedCount_class'])

train_df.drop('instkind_nan', axis=1, inplace=True)
test_df.drop('instkind_nan', axis=1, inplace=True)

train_df.set_index('inst_id', inplace=True)
test_df.set_index('inst_id', inplace=True)

In [3]:
train_lr = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Train_lr.csv')

train_lr['ownerChange'] = round(train_lr['ownerChange'])
train_lr['bedCount_class'] = round(train_lr['bedCount_class'])

train_lr.set_index('inst_id', inplace=True)

train_lr.head()

,sido_choongchung,sido_gyeongsang,sido_jeonra,instkind_nursing_hospital,instkind_traditional_hospital,revenue1,sga1,salary1,noe1,interest1,...,profit2,inventoryAsset2,OnonCAsset2,shortLoan2,NCLiabilities2,longLoan2,surplus2,ownerChange,bedCount_class,OC
inst_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,0.0,22.162515,22.099796,21.433189,16.539187,16.398469,...,19.152229,16.425505,18.798422,0.000000,20.099863,19.781657,20.963246,0.0,3.0,1
3,0.0,1.0,0.0,0.0,0.0,23.980280,23.481393,19.521179,16.823799,15.443340,...,17.025473,16.044684,15.372413,9.180719,18.812726,17.568559,12.258481,0.0,4.0,1
4,0.0,0.0,0.0,1.0,0.0,20.727778,19.918561,19.507228,10.308986,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,4.0,1
7,0.0,0.0,0.0,0.0,0.0,25.006954,24.981392,24.182293,20.954034,20.902985,...,20.546991,20.934483,22.088313,22.944579,23.755157,23.437961,22.769775,0.0,4.0,1
9,0.0,1.0,0.0,0.0,0.0,24.615974,24.587275,23.920337,21.073281,20.991191,...,17.301128,19.908536,20.486709,23.590810,23.784786,23.370010,22.923325,0.0,3.0,1


In [4]:
train_dt = pd.read_csv('D:/숙탯 2기/2022-여름-캐글/최종데이터/Train_dt.csv')
train_dt.drop('instkind_nan', axis=1, inplace=True)

train_dt.head()

,inst_id,instkind_hospital,instkind_traditional_clinic,revenue1,salescost1,sga1,salary1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset2,receivableL2,employee2,OC
0,1,0.0,0.0,22.162515,0.000000,22.099796,21.433189,20.735886,20.720935,19.968362,16.425505,0.0,4.174387,1
1,3,0.0,0.0,23.980280,9.659782,23.481393,19.521179,19.074269,19.037270,10.917985,16.044684,0.0,6.701960,1
2,4,0.0,0.0,20.727778,20.060616,19.918561,19.507228,19.422937,19.351593,15.906875,0.000000,0.0,0.693147,1
3,7,0.0,0.0,25.006954,0.000000,24.981392,24.182293,23.291406,23.168630,0.000000,20.934483,0.0,6.498282,1
4,9,0.0,0.0,24.615974,0.000000,24.587275,23.920337,22.566524,22.493677,22.134087,19.908536,0.0,5.288267,1


# 1. 67개 columns

### 1) GridSearchCV

In [7]:
X = train_df.drop('OC', axis=1)
y = train_df['OC']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

smote = SMOTE(random_state=42)
X_tr_over, y_tr_over = smote.fit_resample(X_train, y_train)

In [ ]:
params = {"max_depth" : list(np.arange(4, 16, 2)),
          "n_estimators" : [1000, 5000, 10000],
          'learning_rate' : list(np.arange(0.0, 0.3, 0.05)),
          'random_strength' : [0, 100],
          "l2_leaf_reg" : [1e-8,3e-5],
          'bagging_temperature' : list(np.arange(0.01, 100.00, 10)),
          "min_child_samples" : [5, 100],
          "max_bin" : [200, 500],
          'od_type' : ['IncToDec', 'Iter']
         }

kfold = KFold(n_splits=5, shuffle = True, random_state=0)

# AdaBoostClassifier 객체 생성 후 GridSearchCV 수행
clf1 = CatBoostClassifier()
grid_cv = GridSearchCV(clf1, param_grid = params, cv = kfold, n_jobs = -1)
grid_cv.fit(X_tr_over, y_tr_over)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
def clf_eval(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차행렬')
    classificationSummary(y_test, pred)
    print()
    print('정확도 : {:.4f}\n정밀도 : {:.4f}\n재현율 : {:.4f}'.format(accuracy, precision, recall))
    print('\nAUC : {:.4f}'.format(roc_auc_score(y_test, pred)))

In [ ]:
best_model = grid_cv.best_estimator_  # 최적의 하이퍼 파라미터로 모델 생성
pred = best_model.predict(X_val)

clf_eval(y_val, pred)

In [54]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.6931200	test: 0.6931186	test1: 0.6930648	best: 0.6930648 (0)	total: 908us	remaining: 908ms
100:	learn: 0.5355416	test: 0.5355828	test1: 0.5368851	best: 0.5349755 (94)	total: 104ms	remaining: 928ms
200:	learn: 0.3892536	test: 0.3892705	test1: 0.4325137	best: 0.4304546 (194)	total: 244ms	remaining: 969ms
300:	learn: 0.2939765	test: 0.2939837	test1: 0.3748481	best: 0.3746073 (299)	total: 386ms	remaining: 896ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.3640117077
bestIteration = 345

Shrink model to first 346 iterations.

Confusion Matrix (Accuracy 0.9344)

       Prediction
Actual  0  1
     0  1  3
     1  1 56

AUC: 0.6162280701754386


### 2) RandomizedSearchCV

In [71]:
def objective2(trial):
    param = {
        "random_state":42,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators":trial.suggest_int("n_estimators", 1000, 5000, 10000),
        "max_depth":trial.suggest_int("max_depth", 4, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
        }
        
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
    smote = SMOTE(random_state=42)
    X_tr_over, y_tr_over = smote.fit_resample(X_train, y_train)
        
    cat = CatBoostClassifier(**param)
    cat.fit(X_tr_over, y_tr_over,
            eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
            early_stopping_rounds=35,
            verbose=100)
    pred = cat.predict(X_val)
    
    return roc_auc_score(y_val, cat.predict(X_val))

In [57]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective2, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-08-29 02:25:42,222] A new study created in memory with name: cat_parameter_opt


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 837us	remaining: 837ms
100:	learn: 0.3369154	test: 0.3369296	test1: 0.4834807	best: 0.4834807 (100)	total: 100ms	remaining: 893ms


[I 2022-08-29 02:25:42,759] Trial 0 finished with value: 0.47368421052631576 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.47368421052631576.


200:	learn: 0.1711298	test: 0.1711573	test1: 0.4289440	best: 0.4228456 (184)	total: 235ms	remaining: 934ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4228456403
bestIteration = 184

Shrink model to first 185 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 592us	remaining: 592ms


[I 2022-08-29 02:25:43,307] Trial 1 finished with value: 0.49122807017543857 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.49122807017543857.


...

0:	learn: 0.6447127	test: 0.6447165	test1: 0.6211090	best: 0.6211090 (0)	total: 1.82ms	remaining: 1.82s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.3611500303
bestIteration = 31

Shrink model to first 32 iterations.


[I 2022-08-29 02:25:57,557] Trial 8 finished with value: 0.5986842105263157 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 8 with value: 0.5986842105263157.


0:	learn: 0.6208398	test: 0.6208606	test1: 0.5474205	best: 0.5474205 (0)	total: 1.27ms	remaining: 1.27s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.3561231893
bestIteration = 37

Shrink model to first 38 iterations.
0:	learn: 0.6514114	test: 0.6514387	test1: 0.6759013	best: 0.6759013 (0)	total: 140ms	remaining: 2m 20s


[I 2022-08-29 02:26:07,069] Trial 9 finished with value: 0.49122807017543857 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 8 with value: 0.5986842105263157.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.3293455855
bestIteration = 40

Shrink model to first 41 iterations.
Best Score: 0.5986842105263157
Best trial {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}


In [58]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.6926854	test: 0.6926898	test1: 0.6917554	best: 0.6917554 (0)	total: 1.48ms	remaining: 1.48s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4045871851
bestIteration = 17

Shrink model to first 18 iterations.

Confusion Matrix (Accuracy 0.8689)

       Prediction
Actual  0  1
     0  0  4
     1  4 53

AUC: 0.4649122807017544


# 2) Train_lr

In [59]:
X = train_lr.drop('OC', axis=1)
y = train_lr['OC']

### 1) accuracy 기준 튜닝

In [61]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective1, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-08-29 02:28:02,976] A new study created in memory with name: cat_parameter_opt
[I 2022-08-29 02:28:03,327] Trial 0 finished with value: 0.8360655737704918 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.8360655737704918.


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 647us	remaining: 647ms
100:	learn: 0.6386989	test: 0.6387319	test1: 0.6292737	best: 0.6267356 (91)	total: 56.4ms	remaining: 502ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.5620827876
bestIteration = 139

Shrink model to first 140 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 591us	remaining: 591ms
100:	learn: 0.6368070	test: 0.6368373	test1: 0.6324273	best: 0.6322432 (99)	total: 66.4ms	remaining: 591ms
200:	learn: 0.5852625	test: 0.5853071	test1: 0.5831538	best: 0.5825012 (192)	total: 144ms	remaining: 572ms


[I 2022-08-29 02:28:03,973] Trial 1 finished with value: 0.8360655737704918 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.8360655737704918.


300:	learn: 0.5382127	test: 0.5382513	test1: 0.5361721	best: 0.5350102 (292)	total: 221ms	remaining: 514ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.5205554758
bestIteration = 351

Shrink model to first 352 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 574us	remaining: 573ms
100:	learn: 0.3449660	test: 0.3449910	test1: 0.4050287	best: 0.4048036 (99)	total: 90.8ms	remaining: 808ms


[I 2022-08-29 02:28:04,475] Trial 2 finished with value: 0.9180327868852459 and parameters: {'learning_rate': 0.08012737503998542, 'bagging_temperature': 0.03613894271216528, 'n_estimators': 1000, 'max_depth': 7, 'random_strength': 37, 'colsample_bylevel': 0.028580510658069373, 'l2_leaf_reg': 2.355742708217648e-05, 'min_child_samples': 24, 'max_bin': 354, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.9180327868852459.


...


Shrink model to first 76 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 807us	remaining: 806ms


[I 2022-08-29 02:28:06,790] Trial 8 finished with value: 0.9016393442622951 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.9180327868852459.


100:	learn: 0.1512638	test: 0.1513001	test1: 0.2982934	best: 0.2982934 (100)	total: 104ms	remaining: 927ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2800631159
bestIteration = 105

Shrink model to first 106 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 661us	remaining: 660ms
100:	learn: 0.1639834	test: 0.1640128	test1: 0.3133246	best: 0.3115347 (96)	total: 1.26s	remaining: 11.2s
200:	learn: 0.0479668	test: 0.0480019	test1: 0.2866364	best: 0.2820036 (192)	total: 2.78s	remaining: 11.1s


[I 2022-08-29 02:28:10,205] Trial 9 finished with value: 0.9016393442622951 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 2 with value: 0.9180327868852459.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2820036275
bestIteration = 192

Shrink model to first 193 iterations.
Best Score: 0.9180327868852459
Best trial {'learning_rate': 0.08012737503998542, 'bagging_temperature': 0.03613894271216528, 'n_estimators': 1000, 'max_depth': 7, 'random_strength': 37, 'colsample_bylevel': 0.028580510658069373, 'l2_leaf_reg': 2.355742708217648e-05, 'min_child_samples': 24, 'max_bin': 354, 'od_type': 'IncToDec'}


In [62]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.6322141	test: 0.6322349	test1: 0.6517800	best: 0.6517800 (0)	total: 5.11ms	remaining: 5.11s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4263870123
bestIteration = 30

Shrink model to first 31 iterations.

Confusion Matrix (Accuracy 0.8852)

       Prediction
Actual  0  1
     0  0  4
     1  3 54

AUC: 0.47368421052631576


### 2) AUC 기준 튜닝

In [65]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective2, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-08-29 02:29:12,318] A new study created in memory with name: cat_parameter_opt
[I 2022-08-29 02:29:12,702] Trial 0 finished with value: 0.6798245614035088 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.6798245614035088.


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 641us	remaining: 640ms
100:	learn: 0.6386989	test: 0.6387319	test1: 0.6292737	best: 0.6267356 (91)	total: 66.4ms	remaining: 591ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.5620827876
bestIteration = 139

Shrink model to first 140 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 704us	remaining: 704ms
100:	learn: 0.6368070	test: 0.6368373	test1: 0.6324273	best: 0.6322432 (99)	total: 69.8ms	remaining: 621ms
200:	learn: 0.5852625	test: 0.5853071	test1: 0.5831538	best: 0.5825012 (192)	total: 140ms	remaining: 555ms


[I 2022-08-29 02:29:13,363] Trial 1 finished with value: 0.6798245614035088 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.6798245614035088.


300:	learn: 0.5382127	test: 0.5382513	test1: 0.5361721	best: 0.5350102 (292)	total: 212ms	remaining: 492ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.5205554758
bestIteration = 351

Shrink model to first 352 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 820us	remaining: 820ms
100:	learn: 0.3449660	test: 0.3449910	test1: 0.4050287	best: 0.4048036 (99)	total: 124ms	remaining: 1.1s


[I 2022-08-29 02:29:13,957] Trial 2 finished with value: 0.6074561403508771 and parameters: {'learning_rate': 0.08012737503998542, 'bagging_temperature': 0.03613894271216528, 'n_estimators': 1000, 'max_depth': 7, 'random_strength': 37, 'colsample_bylevel': 0.028580510658069373, 'l2_leaf_reg': 2.355742708217648e-05, 'min_child_samples': 24, 'max_bin': 354, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.6798245614035088.


...

Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.267094811
bestIteration = 75

Shrink model to first 76 iterations.


[I 2022-08-29 02:29:16,482] Trial 8 finished with value: 0.5986842105263157 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 3 with value: 0.7236842105263157.


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 755us	remaining: 755ms
100:	learn: 0.1512638	test: 0.1513001	test1: 0.2982934	best: 0.2982934 (100)	total: 77.3ms	remaining: 688ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2800631159
bestIteration = 105

Shrink model to first 106 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 553us	remaining: 553ms
100:	learn: 0.1639834	test: 0.1640128	test1: 0.3133246	best: 0.3115347 (96)	total: 1.27s	remaining: 11.3s
200:	learn: 0.0479668	test: 0.0480019	test1: 0.2866364	best: 0.2820036 (192)	total: 2.77s	remaining: 11s


[I 2022-08-29 02:29:19,814] Trial 9 finished with value: 0.4824561403508772 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 3 with value: 0.7236842105263157.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.2820036275
bestIteration = 192

Shrink model to first 193 iterations.
Best Score: 0.7236842105263157
Best trial {'learning_rate': 0.07896186801026692, 'bagging_temperature': 0.04809461967501574, 'n_estimators': 1000, 'max_depth': 4, 'random_strength': 95, 'colsample_bylevel': 0.0923915031962725, 'l2_leaf_reg': 2.425383647001267e-05, 'min_child_samples': 34, 'max_bin': 229, 'od_type': 'IncToDec'}


In [66]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.6516007	test: 0.6516079	test1: 0.6769923	best: 0.6769923 (0)	total: 1.97ms	remaining: 1.97s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.3483835881
bestIteration = 21

Shrink model to first 22 iterations.

Confusion Matrix (Accuracy 0.8525)

       Prediction
Actual  0  1
     0  1  3
     1  6 51

AUC: 0.5723684210526316


# 3) Train_dt

In [69]:
X = train_dt.drop('OC', axis=1)
y = train_dt['OC']

### 1) accuracy 기준

In [72]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective1, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-08-29 02:31:34,990] A new study created in memory with name: cat_parameter_opt
[I 2022-08-29 02:31:35,324] Trial 0 finished with value: 0.45901639344262296 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.45901639344262296.


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 560us	remaining: 560ms
100:	learn: 0.6601862	test: 0.6602051	test1: 0.6759648	best: 0.6752585 (95)	total: 55ms	remaining: 489ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.6722004338
bestIteration = 120

Shrink model to first 121 iterations.


[I 2022-08-29 02:31:35,612] Trial 1 finished with value: 0.7377049180327869 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.7377049180327869.


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 670us	remaining: 670ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.6923058372
bestIteration = 45

Shrink model to first 46 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 1.16ms	remaining: 1.16s
100:	learn: 0.5337948	test: 0.5338160	test1: 0.5786219	best: 0.5721180 (85)	total: 87.2ms	remaining: 776ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4901044457
bestIteration = 132


[I 2022-08-29 02:31:36,020] Trial 2 finished with value: 0.9016393442622951 and parameters: {'learning_rate': 0.08012737503998542, 'bagging_temperature': 0.03613894271216528, 'n_estimators': 1000, 'max_depth': 7, 'random_strength': 37, 'colsample_bylevel': 0.028580510658069373, 'l2_leaf_reg': 2.355742708217648e-05, 'min_child_samples': 24, 'max_bin': 354, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.9016393442622951.


...

0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 644us	remaining: 644ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.6521068126
bestIteration = 12

Shrink model to first 13 iterations.


[I 2022-08-29 02:31:37,683] Trial 8 finished with value: 0.7540983606557377 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 5 with value: 0.9344262295081968.


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 659us	remaining: 659ms
100:	learn: 0.3374654	test: 0.3374837	test1: 0.4926306	best: 0.4709431 (96)	total: 76.1ms	remaining: 678ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4709430723
bestIteration = 96

Shrink model to first 97 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 506us	remaining: 506ms
100:	learn: 0.5320434	test: 0.5320777	test1: 0.5711520	best: 0.5711520 (100)	total: 83.8ms	remaining: 746ms
200:	learn: 0.4638949	test: 0.4639259	test1: 0.5471357	best: 0.5452097 (193)	total: 159ms	remaining: 633ms


[I 2022-08-29 02:31:38,188] Trial 9 finished with value: 0.8524590163934426 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 5 with value: 0.9344262295081968.


Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.5276831411
bestIteration = 236

Shrink model to first 237 iterations.
Best Score: 0.9344262295081968
Best trial {'learning_rate': 0.27051668818999286, 'bagging_temperature': 12.60466458564947, 'n_estimators': 1000, 'max_depth': 16, 'random_strength': 90, 'colsample_bylevel': 0.03961867790406585, 'l2_leaf_reg': 2.7657008308343274e-05, 'min_child_samples': 13, 'max_bin': 258, 'od_type': 'Iter'}


In [73]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.4215680	test: 0.4216015	test1: 0.5598141	best: 0.5598141 (0)	total: 50.5ms	remaining: 50.4s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.3980716115
bestIteration = 2

Shrink model to first 3 iterations.

Confusion Matrix (Accuracy 0.8033)

       Prediction
Actual  0  1
     0  2  2
     1 10 47

AUC: 0.6622807017543859


### 2) AUC 기준

In [74]:
sampler = TPESampler(seed=42)

study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler)

study.optimize(objective2, n_trials=10)

print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

[I 2022-08-29 02:32:16,763] A new study created in memory with name: cat_parameter_opt
[I 2022-08-29 02:32:17,078] Trial 0 finished with value: 0.47807017543859653 and parameters: {'learning_rate': 0.03574712922600244, 'bagging_temperature': 63.512210106407046, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 60, 'colsample_bylevel': 0.014322493718230255, 'l2_leaf_reg': 4.688275664882717e-06, 'min_child_samples': 10, 'max_bin': 460, 'od_type': 'Iter'}. Best is trial 0 with value: 0.47807017543859653.


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 562us	remaining: 561ms
100:	learn: 0.6601862	test: 0.6602051	test1: 0.6759648	best: 0.6752585 (95)	total: 52.4ms	remaining: 467ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.6722004338
bestIteration = 120

Shrink model to first 121 iterations.


[I 2022-08-29 02:32:17,359] Trial 1 finished with value: 0.5109649122807017 and parameters: {'learning_rate': 0.010725209743171996, 'bagging_temperature': 75.7947995334801, 'n_estimators': 1000, 'max_depth': 14, 'random_strength': 21, 'colsample_bylevel': 0.015199348301309814, 'l2_leaf_reg': 5.51030125050448e-06, 'min_child_samples': 34, 'max_bin': 357, 'od_type': 'IncToDec'}. Best is trial 1 with value: 0.5109649122807017.


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 606us	remaining: 605ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.6923058372
bestIteration = 45

Shrink model to first 46 iterations.


[I 2022-08-29 02:32:17,762] Trial 2 finished with value: 0.5986842105263157 and parameters: {'learning_rate': 0.08012737503998542, 'bagging_temperature': 0.03613894271216528, 'n_estimators': 1000, 'max_depth': 7, 'random_strength': 37, 'colsample_bylevel': 0.028580510658069373, 'l2_leaf_reg': 2.355742708217648e-05, 'min_child_samples': 24, 'max_bin': 354, 'od_type': 'IncToDec'}. Best is trial 2 with value: 0.5986842105263157.


...

0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 671us	remaining: 671ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.6521068126
bestIteration = 12

Shrink model to first 13 iterations.


[I 2022-08-29 02:32:19,541] Trial 8 finished with value: 0.5197368421052632 and parameters: {'learning_rate': 0.18832519048593593, 'bagging_temperature': 3.1130959561221245, 'n_estimators': 1000, 'max_depth': 8, 'random_strength': 6, 'colsample_bylevel': 0.020463613363481608, 'l2_leaf_reg': 9.762247827582143e-06, 'min_child_samples': 75, 'max_bin': 391, 'od_type': 'IncToDec'}. Best is trial 7 with value: 0.6447368421052632.


0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 612us	remaining: 612ms
100:	learn: 0.3374654	test: 0.3374837	test1: 0.4926306	best: 0.4709431 (96)	total: 58.2ms	remaining: 518ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.4709430723
bestIteration = 96

Shrink model to first 97 iterations.
0:	learn: 0.6931464	test: 0.6931472	test1: 0.6931472	best: 0.6931472 (0)	total: 591us	remaining: 590ms
100:	learn: 0.5320434	test: 0.5320777	test1: 0.5711520	best: 0.5711520 (100)	total: 93.1ms	remaining: 829ms


[I 2022-08-29 02:32:20,072] Trial 9 finished with value: 0.6885964912280702 and parameters: {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}. Best is trial 9 with value: 0.6885964912280702.


200:	learn: 0.4638949	test: 0.4639259	test1: 0.5471357	best: 0.5452097 (193)	total: 179ms	remaining: 711ms
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.5276831411
bestIteration = 236

Shrink model to first 237 iterations.
Best Score: 0.6885964912280702
Best trial {'learning_rate': 0.015019490572374374, 'bagging_temperature': 7.128188058401368, 'n_estimators': 1000, 'max_depth': 13, 'random_strength': 56, 'colsample_bylevel': 0.05901564798023389, 'l2_leaf_reg': 1.4818929934968078e-05, 'min_child_samples': 55, 'max_bin': 328, 'od_type': 'Iter'}


In [75]:
cat = CatBoostClassifier(**study.best_params)
cat.fit(X_tr_over, y_tr_over,
        eval_set=[(X_tr_over, y_tr_over), (X_val, y_val)],
        early_stopping_rounds=35,
        verbose=100)

print()
classificationSummary(y_val, cat.predict(X_val))
print('\nAUC:', roc_auc_score(y_val, cat.predict(X_val)))

0:	learn: 0.6558307	test: 0.6558513	test1: 0.6777402	best: 0.6777402 (0)	total: 181ms	remaining: 3m 1s
Stopped by overfitting detector  (35 iterations wait)

bestTest = 0.3320834414
bestIteration = 42

Shrink model to first 43 iterations.

Confusion Matrix (Accuracy 0.9180)

       Prediction
Actual  0  1
     0  0  4
     1  1 56

AUC: 0.49122807017543857
